<a href="https://colab.research.google.com/github/rbdus0715/Machine-Learning/blob/main/study/sklearn/ensemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **앙상블**

### (1) 앙상블이란
- 여러 개의 분류기를 생성하고 그 예측을 결합하여 정확한 최종 예측을 도출하는 기법을 말한다.
- 이미지, 영상, 음성 등의 비정형 데이터의 분류는 딥러닝이 뛰어난 성능을 보이지만, 대부분의 정형 데이터 분류 시에는 앙상블이 뛰어난 성능을 보인다.
> 비정형 데이터 : 표준화된 형식이고, 구조가 잘 정의되어 있으며, 데이터 모델을 준수하고, 지속적인 순서를 따르고, 인간과 프로그램이 쉽게 액세스할 수 있는 데이터

- 유명한 앙상블 알고리즘
    - 랜덤 포레스트
    - 그래디언트 부스팅
- 더욱 발전
    -  XGBoost
    - LightGBM


### (2) 학습 유형
- Voting 방식 : 서로 다른 분류기를 결합하여 투표를 통해 최종 예측 결과를 결정
- Bagging 방식 : 같은 분류기 여러개를 결합하여 투표를 통해 최종 예측 결과를 결정
    - 개별 분류기에 할당된 학습 데이터는 원본 학습 데이터를 샘플링해 추출 >> 부트스트래핑 분할 방식
    - 개별 분류기가 샘플링된 데이터 세트에 대해 학습을 통해 개별적인 예측을 수행한 결과를 보팅을 통해 선정
    - **교차 검증이 데이터 세트간에 중첩을 허용하지 않는 것과 다르게 배깅 방식은 중첩을 허용**
- Boosting 방식 : 여러 개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록, 다음 분류기에게는 가중치를 부여하면서 학습과 예측을 진행


## **보팅 - 하드 보팅, 소프트 보팅**
[api](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier)
- 하드 보팅 : 다수결 원칙
- 소프트 보팅 : 결정 확률 평균 (일반적으로 많이 사용)

In [3]:
import pandas as pd

# 보팅 방식의 앙상블 클래스
from sklearn.ensemble import VotingClassifier

# 사용될 분류기들
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

df = pd.DataFrame(cancer.data, cancer.target, columns=cancer.feature_names)
df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
0,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
0,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


앙상블 훈련 & 예측 성능

In [4]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)

0.9473684210526315


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


각각의 분류기 예측 성능

In [5]:
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print(accuracy)

0.9385964912280702
0.9385964912280702


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
